In [16]:
import nltk
nltk.download('webtext')
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import webtext
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer, LancasterStemmer
from nltk.stem import WordNetLemmatizer
import string
from collections import Counter


[nltk_data] Downloading package webtext to /root/nltk_data...
[nltk_data]   Unzipping corpora/webtext.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [17]:
import pandas as pd

documents = webtext.fileids()
data = []

for doc in documents:
    text = webtext.raw(doc)
    tokens = word_tokenize(text)
    lowercase_tokens = [token.lower() for token in tokens]
    stop_words = [token for token in lowercase_tokens if token in stopwords.words('english')]
    uppercase_tokens = [token for token in tokens if token.isupper()]

    data.append([doc, len(stop_words), len(tokens), len(uppercase_tokens)])

df = pd.DataFrame(data, columns=["Document_Name", "Stop_Words", "Tokens", "Uppercase_Tokens"])
print("5 premières lignes du DataFrame :")
print(df.head())
print("\n5 dernières lignes du DataFrame :")
print(df.tail())


5 premières lignes du DataFrame :
   Document_Name  Stop_Words  Tokens  Uppercase_Tokens
0    firefox.txt       26545   96116              2079
1      grail.txt        4380   16460              1760
2  overheard.txt       71425  210101              7460
3    pirates.txt        6950   21987              1771
4    singles.txt         727    4466               410

5 dernières lignes du DataFrame :
   Document_Name  Stop_Words  Tokens  Uppercase_Tokens
1      grail.txt        4380   16460              1760
2  overheard.txt       71425  210101              7460
3    pirates.txt        6950   21987              1771
4    singles.txt         727    4466               410
5       wine.txt        9243   33446               676


In [18]:
df['Tokens'] = df['Document_Name'].apply(lambda doc: [token for token in word_tokenize(webtext.raw(doc)) if token.lower() not in stopwords.words('english')])
df['Tokens'] = df['Tokens'].apply(lambda tokens: [token.lower() for token in tokens])


In [19]:
df

,Document_Name,Stop_Words,Tokens,Uppercase_Tokens
0,firefox.txt,26545,"[cookie, manager, :, ``, n't, allow, sites, se...",2079
1,grail.txt,4380,"[scene, 1, :, [, wind, ], [, clop, clop, clop,...",1760
2,overheard.txt,71425,"[white, guy, :, ,, plans, evening, ?, asian, g...",7460
3,pirates.txt,6950,"[pirates, carribean, :, dead, man, 's, chest, ...",1771
4,singles.txt,727,"[25, sexy, male, ,, seeks, attrac, older, sing...",410
5,wine.txt,9243,"[lovely, delicate, ,, fragrant, rhone, wine, ....",676


In [20]:
# filtered_tokens = [token.lower() for token in tokens if token.lower() not in stopwords.words('english')]


In [21]:
all_tokens = [token for tokens in df['Tokens'] for token in tokens]
token_freq = FreqDist(all_tokens)
most_common_tokens = token_freq.most_common(10)
print("Les 10 tokens les plus fréquents :")
print(most_common_tokens)


Les 10 tokens les plus fréquents :
[('.', 16479), (':', 14328), (',', 12427), ('?', 4743), ('!', 4417), ('*', 3944), ('#', 3777), ("n't", 3481), ("'s", 3198), ('girl', 2955)]


In [22]:
# Vous pouvez supprimer les 10 tokens les moins fréquents de chaque document en utilisant "most_common" ou "remove".
# Par exemple, pour supprimer les 10 tokens les moins fréquents de chaque document :
least_common_tokens = token_freq.most_common()[:-11:-1]
df['Tokens'] = df['Tokens'].apply(lambda tokens: [token for token in tokens if token not in [t[0] for t in least_common_tokens]])


In [23]:
def remove_digits_and_punctuation(text):
    text = ''.join([char for char in text if char not in string.punctuation and not char.isdigit()])
    return text


In [24]:
import inflect

def convert_numbers_to_words(text):
    p = inflect.engine()
    words = text.split()
    words = [p.number_to_words(word) if word.isdigit() else word for word in words]
    return ' '.join(words)


In [25]:
!pip install emoji

In [26]:
import emoji


In [27]:
def compter_emojis(texte):
  """
  Compte le nombre d'emojis dans un texte.

  Args:
    texte: Le texte à analyser.

  Returns:
    Le nombre d'emojis dans le texte.
  """

  compteur = 0
  for caractère in texte:
    if caractère in emoji.EMOJI_DATA:
      compteur += 1
  return compteur


In [28]:
texte = "Ceci est un texte avec des emojis 😀 😊 😍 😇 😉 😊 😊"

compteur = compter_emojis(texte)

print(compteur)


7


In [29]:
import re

In [30]:
emojis = re.findall(r"[\U0001F600-\U0001F64F]", texte)

for emoji in emojis:
    texte = texte.replace(emoji, "")

print(texte)

Ceci est un texte avec des emojis       


In [31]:
from nltk.stem import PorterStemmer
porter_stemmer = PorterStemmer()
df['Porter Stemmed Tokens'] = df['Tokens'].apply(lambda tokens: [porter_stemmer.stem(token) for token in tokens])


In [32]:
from nltk.stem.lancaster import LancasterStemmer
lancaster_stemmer = LancasterStemmer()
df['Lancaster Stemmed Tokens'] = df['Tokens'].apply(lambda tokens: [lancaster_stemmer.stem(token) for token in tokens])


In [35]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()
df['Lemmatized Tokens'] = df['Tokens'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])


[nltk_data] Downloading package wordnet to /root/nltk_data...


**Qu'est-ce que WordNet ?**

WordNet est une base de données lexicale de la langue anglaise qui organise les mots en synsets (ensembles de synonymes) et établit des relations sémantiques entre les mots. Il est utilisé pour des tâches telles que la recherche sémantique, la lemmatisation et la désambiguïsation lexicale. WordNet est principalement destiné à la langue anglaise.

**Quelles langues sont supportées ?**

WordNet est principalement destiné à l'anglais, mais il existe également des versions adaptées à d'autres langues telles que WordNet pour le français (French WordNet), WordNet pour l'espagnol (EuroWordNet), etc. Cependant, la couverture et la qualité des informations peuvent varier en fonction de la langue.